* BPE와 유사한 알고리즘인 Wordpiece Model 채택
* 패키지를 통해 간단하게 단어들을 서브워드로 분리 가능

In [1]:
import tensorflow
tensorflow.__version__

'2.3.0'

In [2]:
!pip install tensorflow_datasets

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 5.3/5.3 MB 19.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 146.5/146.5 kB ? eta 0:00:00
     ---------------------------------------- 101.4/101.4 kB ? eta 0:00:00
     ---------------------------------------- 52.3/52.3 kB ? eta 0:00:00
     ------------------------------------- 223.0/223.0 kB 13.3 MB/s eta 0:00:00
     ------------------------------------- 904.4/904.4 kB 19.0 MB/s eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21486 sha256=e90f0ba07bbcbb6fb8d5e265b8d60c6e968f2fcb08f8ff97cb781ab2d4707a53
  Stored in directory: c:\users\tada2\appdata\local\pip\cache\wheels\54\aa\01\724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
Successfully built promise


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.3.0 requires gast==0.3.3, but you have gast 0.4.0 which is incompatible.
tensorflow 2.3.0 requires numpy<1.19.0,>=1.16.0, but you have numpy 1.21.5 which is incompatible.
tensorflow 2.3.0 requires scipy==1.4.1, but you have scipy 1.9.3 which is incompatible.
tensorflow 2.3.0 requires tensorflow-estimator<2.4.0,>=2.3.0, but you have tensorflow-estimator 2.6.0 which is incompatible.
tensorboard 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 1. IMDB 리뷰 토큰화

In [3]:
import pandas as pd
import urllib.request
import tensorflow_datasets as tfds

In [4]:
# 데이터 로드 및 저장
train_df = pd.read_csv('IMDB_Reviews.csv')
train_df['review']

0        My family and I normally do not watch local mo...
1        Believe it or not, this was at one time the wo...
2        After some internet surfing, I found the "Home...
3        One of the most unheralded great works of anim...
4        It was the Sixties, and anyone with long hair ...
                               ...                        
49995    the people who came up with this are SICK AND ...
49996    The script is so so laughable... this in turn,...
49997    "So there's this bride, you see, and she gets ...
49998    Your mind will not be satisfied by this nobud...
49999    The chaser's war on everything is a weekly sho...
Name: review, Length: 50000, dtype: object

* tfds.deprecated.text.SubwordTextEncoder.build_from_corpus의 인자로 토큰화할 데이터를 넣어 줌
* 이 작업을 통해 서브워드로 이루어진 단어집합을 생성하고, 각 서브워드에 고유한 정수 부여

In [5]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    train_df['review'], target_vocab_size=2**13)

* .subwords를 통해 토큰화된 서브워드 확인 가능

In [6]:
print(tokenizer.subwords[:100])

['the_', ', ', '. ', 'a_', 'and_', 'of_', 'to_', 's_', 'is_', 'br', 'in_', 'I_', 'that_', 'this_', 'it_', ' /><', ' />', 'was_', 'The_', 't_', 'as_', 'with_', 'for_', '.<', 'on_', 'but_', 'movie_', 'are_', ' (', 'have_', 'his_', 'film_', 'not_', 'be_', 'you_', 'ing_', ' "', 'ed_', 'it', 'd_', 'an_', 'at_', 'by_', 'he_', 'one_', 'who_', 'from_', 'y_', 'or_', 'e_', 'like_', 'all_', '" ', 'they_', 'so_', 'just_', 'has_', ') ', 'about_', 'her_', 'out_', 'This_', 'some_', 'movie', 'ly_', 'film', 'very_', 'more_', 'It_', 'what_', 'would_', 'when_', 'if_', 'good_', 'up_', 'which_', 'their_', 'only_', 'even_', 'my_', 'really_', 'had_', 'can_', 'no_', 'were_', 'see_', '? ', 'she_', 'than_', '! ', 'there_', 'been_', 'get_', 'into_', 'will_', ' - ', 'much_', 'n_', 'because_', 'ing']


In [7]:
# 임의의 20번 인텍스의 샘플 출력, 정수인코딩한 결과 비교
print(train_df['review'][20])

Pretty bad PRC cheapie which I rarely bother to watch over again, and it's no wonder -- it's slow and creaky and dull as a butter knife. Mad doctor George Zucco is at it again, turning a dimwitted farmhand in overalls (Glenn Strange) into a wolf-man. Unfortunately, the makeup is virtually non-existent, consisting only of a beard and dimestore fangs for the most part. If it were not for Zucco and Strange's presence, along with the cute Anne Nagel, this would be completely unwatchable. Strange, who would go on to play Frankenstein's monster for Unuiversal in two years, does a Lenny impression from "Of Mice and Men", it seems.<br /><br />*1/2 (of Four)


In [8]:
print('Tokenized sample question: {}'.format(tokenizer.encode(train_df['review'][20])))

Tokenized sample question: [1590, 4162, 132, 7107, 1892, 2983, 578, 76, 12, 4632, 3422, 7, 160, 175, 372, 2, 5, 39, 8051, 8, 84, 2652, 497, 39, 8051, 8, 1374, 5, 3461, 2012, 48, 5, 2263, 21, 4, 2992, 127, 4729, 711, 3, 1391, 8044, 3557, 1277, 8102, 2154, 5681, 9, 42, 15, 372, 2, 3773, 4, 3502, 2308, 467, 4890, 1503, 11, 3347, 1419, 8127, 29, 5539, 98, 6099, 58, 94, 4, 1388, 4230, 8057, 213, 3, 1966, 2, 1, 6700, 8044, 9, 7069, 716, 8057, 6600, 2, 4102, 36, 78, 6, 4, 1865, 40, 5, 3502, 1043, 1645, 8044, 1000, 1813, 23, 1, 105, 1128, 3, 156, 15, 85, 33, 23, 8102, 2154, 5681, 5, 6099, 8051, 8, 7271, 1055, 2, 534, 22, 1, 3046, 5214, 810, 634, 8120, 2, 14, 71, 34, 436, 3311, 5447, 783, 3, 6099, 2, 46, 71, 193, 25, 7, 428, 2274, 2260, 6487, 8051, 8, 2149, 23, 1138, 4117, 6023, 163, 11, 148, 735, 2, 164, 4, 5277, 921, 3395, 1262, 37, 639, 1349, 349, 5, 2460, 328, 15, 5349, 8127, 24, 10, 16, 10, 17, 8054, 8061, 8059, 8062, 29, 6, 6607, 8126, 8053]


In [9]:
# train_df에 존재하는 문장 중 일부 발췌
sample_string =  "It's mind-blowing to me that this film was even made."

# 인코딩 결과를 tokenized_string에 저장
tokenized_string = tokenizer.encode(sample_string)
print('정수 인코딩 후의 문장: {}'.format(tokenized_string))

정수 인코딩 후의 문장: [137, 8051, 8, 910, 8057, 2169, 36, 7, 103, 13, 14, 32, 18, 79, 681, 8058]


In [10]:
# 다시 디코딩
original_string = tokenizer.decode(tokenized_string)
print('기존 문장 : {}'.format(original_string))

기존 문장 : It's mind-blowing to me that this film was even made.


* .vocab_size를 통해 단어집합의 크기 확인 가능

In [11]:
print(tokenizer.vocab_size)

8268


In [12]:
# 디코딩 결과를 병렬적으로 나열하여 각 단어와 맵핑된 정수 확인
for ts in tokenized_string:
    print('{} ---> {}'.format(ts, tokenizer.decode([ts])))

137 ---> It
8051 ---> '
8 ---> s 
910 ---> mind
8057 ---> -
2169 ---> blow
36 ---> ing 
7 ---> to 
103 ---> me 
13 ---> that 
14 ---> this 
32 ---> film 
18 ---> was 
79 ---> even 
681 ---> made
8058 ---> .


In [14]:
# 기존 예제 문장 중 even 이라는 단어에 임의로 xyz 글자를 추가
sample_string = "It's mind-blowing to me that this film was evenxyz made."

# 인코딩한 결과를 tokenized_string에 저장
tokenized_string = tokenizer.encode(sample_string)
print('정수 인코딩 후의 문장 : {}'.format(tokenized_string))

정수 인코딩 후의 문장 : [137, 8051, 8, 910, 8057, 2169, 36, 7, 103, 13, 14, 32, 18, 7974, 8132, 8133, 997, 681, 8058]


In [17]:
for token_num in tokenized_string:
    print("{} --> {}".format(token_num, tokenizer.decode([token_num])))

137 --> It
8051 --> '
8 --> s 
910 --> mind
8057 --> -
2169 --> blow
36 --> ing 
7 --> to 
103 --> me 
13 --> that 
14 --> this 
32 --> film 
18 --> was 
7974 --> even
8132 --> x
8133 --> y
997 --> z 
681 --> made
8058 --> .


* x, y, z에 대한 서브워드가 각각 분리되어 추가됨

### 2. 네이버 영화 리뷰 토큰화


In [18]:
import pandas as pd
import tensorflow_datasets as tfds

In [25]:
# 데이터 로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
train_data = pd.read_table('ratings_train.txt')

In [26]:
# Null 값 확인 후 제거
train_data.isnull().sum()

id          0
document    5
label       0
dtype: int64

In [27]:
train_data = train_data.dropna(how='any')
train_data.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [28]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    train_data['document'], target_vocab_size=2**13
)

In [29]:
# 토큰화된 100개의 서브워드 출력
print(tokenizer.subwords[:100])

['. ', '..', '영화', '이_', '...', '의_', '는_', '도_', '다', ', ', '을_', '고_', '은_', '가_', '에_', '.. ', '한_', '너무_', '정말_', '를_', '고', '게_', '영화_', '지', '... ', '진짜_', '이', '다_', '요', '만_', '? ', '과_', '나', '가', '서_', '지_', '로_', '으로_', '아', '어', '....', '음', '한', '수_', '와_', '도', '네', '그냥_', '나_', '더_', '왜_', '이런_', '면_', '기', '하고_', '보고_', '하는_', '서', '좀_', '리', '자', '스', '안', '! ', '에서_', '영화를_', '미', 'ㅋㅋ', '네요', '시', '주', '라', '는', '오', '없는_', '에', '해', '사', '!!', '영화는_', '마', '잘_', '수', '영화가_', '만', '본_', '로', '그_', '지만_', '대', '은', '비', '의', '일', '개', '있는_', '없다', '함', '구', '하']


In [31]:
# encode를 통해 사례 테스트
sample = train_data['document'][20]
print(sample)

나름 심오한 뜻도 있는 듯. 그냥 학생이 선생과 놀아나는 영화는 절대 아님


In [33]:
print(tokenizer.encode(sample))

[669, 4700, 17, 1749, 8, 96, 131, 1, 48, 2239, 4, 7466, 32, 1274, 2655, 7, 80, 749, 1254]


In [34]:
sample = train_data['document'][21]
print(sample)

보면서 웃지 않는 건 불가능하다


In [35]:
# 인코딩한 결과를 tokenized_string에 저장
tokenized_string = tokenizer.encode(sample)
print("정수 인코딩 후의 문장 : {}".format(tokenized_string))

# 다시 디코딩
original_string = tokenizer.decode(tokenized_string)
print('기존 문장 : {}'.format(original_string))

정수 인코딩 후의 문장 : [570, 892, 36, 584, 159, 7091, 201]
기존 문장 : 보면서 웃지 않는 건 불가능하다


In [37]:
for ts in tokenized_string:
    print("{} --> {}".format(ts, tokenizer.decode([ts])))

570 --> 보면서 
892 --> 웃
36 --> 지 
584 --> 않는 
159 --> 건 
7091 --> 불가능
201 --> 하다
